## How to Use this Real time testing file :
Dependencies are listed below, make sure you install them before running the file<br>
In the Part Using our own trained model, set the path of the model .<br>
Make sure you have proper internet connectivity.<br>
Run the complete file.<br>

In [20]:
# Import dependencies
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from gtts import gTTS
language = 'en'
from pygame import mixer

In [21]:
from tempfile import NamedTemporaryFile

In [22]:
mp_holistic = mp.solutions.holistic  # Holistic model
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities

In [23]:
def mediapipe_detection(image, model):
    # COLOR CONVERSION BGR 2 RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # COLOR COVERSION RGB 2 BGR
    return image, results

In [24]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,
                              mp_holistic.FACEMESH_CONTOURS)  # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks,
                              mp_holistic.POSE_CONNECTIONS)  # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS)  # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS)  # Draw right hand connections

In [25]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(
                                  color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(
                                  color=(80, 256, 121), thickness=1, circle_radius=1)
                              )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(
                                  color=(80, 44, 121), thickness=2, circle_radius=2)
                              )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(
                                  color=(121, 44, 250), thickness=2, circle_radius=2)
                              )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(
                                  color=(245, 66, 230), thickness=2, circle_radius=2)
                              )

In [26]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten(
    ) if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten(
    ) if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten(
    ) if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten(
    ) if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

#### Using our trained model

In [27]:
from keras.models import load_model

model = load_model(
    '/Users/krishkhadria/Desktop/python files/sd_model/actions_5_latest_diffmodel1.h5')

In [28]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join(
    '/Users/krishkhadria/Desktop/python files/GestureDetection')

# Actions that we try to detect
actions = np.array(['Hello', 'Bye', 'Thanks', 'Love', 'Shoot'])

# Number of sequences (videos) for each action
no_sequences = 30

# Number of frames in each video sequence
sequence_length = 15

In [29]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [30]:
label_map = {label: num for num, label in enumerate(actions)}

In [31]:
from scipy import stats

In [32]:
colors = []
for _ in range(len(label_map)):
    num = np.random.randint(0, 256)
    colors.append((num, num,num))

In [34]:
# Generate random RGB colors with fixed blue component for each label in the label map
colors = [(np.random.randint(0, 256), np.random.randint(0, 256), 255)
          for _ in range(len(label_map))]


def prob_viz(res, actions, input_frame, colors):
    """
    Visualize the probabilities of actions using circles on the input frame.

    Parameters:
    - res: List of probabilities for each action.
    - actions: List of action labels.
    - input_frame: Original input frame.
    - colors: List of RGB colors corresponding to each action.

    Returns:
    - output_frame: Frame with probability circles and centered labels visualized.
    """
    output_frame = input_frame.copy()

    # Iterate through probabilities and actions to visualize on the frame
    for num, prob in enumerate(res):
        # Calculate the radius based on the probability (spread in the circle)
        radius = int(prob * 50)

        # Draw filled circle as a probability indicator
        cv2.circle(output_frame, (120, 80 + num * 80), radius, colors[num], -1)

        # Add centered text label within the circle
        text_size = cv2.getTextSize(
            actions[num], cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
        text_x = max(120 - text_size[0] // 2, 0)
        text_y = max(80 + num * 80 + text_size[1] // 2, 0)
        cv2.putText(output_frame, actions[num], (text_x, text_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    return output_frame

# Size of the Circle (Radius): The larger the circle, the higher the probability.
# A small circle indicates a low probability, while a larger circle signifies a higher probability.
# Color of the Circle: The color of each circle is randomly assigned but remains consistent for a
# specific action. While not directly representing probability, it helps differentiate between different actions.

In [35]:
# res = np.load(os.path.join(DATA_PATH, action, str(
#     sequence), "{}.npy".format(frame_num)))

In [36]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.5

# Set up video capture
cap = cv2.VideoCapture(0)

# Set up MediaPipe Holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    mixer.init()
    # Initialize a variable to store the last spoken word
    last_spoken_word = None


    while cap.isOpened():
        # Read video feed
        ret, frame = cap.read()

        # Make detections using the Holistic model
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks on the frame
        draw_styled_landmarks(image, results)

        # 2. Prediction logic
        # Extract keypoints from the Holistic model results
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        # Keep only the last 15 frames in the sequence
        sequence = sequence[-15:]

        # Make a prediction using the trained model
        if len(sequence) == 15:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

            # 3. Visualization logic
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5:
                    sentence = sentence[-5:]

                # Visualize probabilities on the image
                image = prob_viz(res, actions, image, colors)

        # Display sentence and probabilities on the frame
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 221, 158), 2, cv2.LINE_AA)

        # Speak the recognized word using gTTS
        if sentence:
            current_word = sentence[-1]

            # Check if the current word is different from the last spoken word
            if current_word != last_spoken_word:
                last_spoken_word = current_word

                # Speak the word using gTTS
                with NamedTemporaryFile(delete=True) as temp_audio:
                    tts = gTTS(text=current_word, lang='en')
                    tts.save(f"{temp_audio.name}.mp3")
                    mixer.music.load(f"{temp_audio.name}.mp3")
                    mixer.music.play()

        # Show the frame
        cv2.imshow('OpenCV Feed', image)

        # Break out of the loop when 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Release video capture resources
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

1/1 [==============================] - 1s 770ms/step
Love
1/1 [==============================] - 0s 20ms/step
Love
1/1 [==============================] - 0s 19ms/step
Love
1/1 [==============================] - 0s 19ms/step
Love
1/1 [==============================] - 0s 19ms/step
Love
1/1 [==============================] - 0s 20ms/step
Love
1/1 [==============================] - 0s 19ms/step
Love
1/1 [==============================] - 0s 18ms/step
Love
1/1 [==============================] - 0s 19ms/step
Love
1/1 [==============================] - 0s 18ms/step
Love
1/1 [==============================] - 0s 19ms/step
Love
1/1 [==============================] - 0s 19ms/step
Love
1/1 [==============================] - 0s 19ms/step
Love
1/1 [==============================] - 0s 18ms/step
Love
1/1 [==============================] - 0s 19ms/step
Love
1/1 [==============================] - 0s 19ms/step
Love
1/1 [==============================] - 0s 20ms/step
Love
1/1 [=========================